# Same as Before

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, learning_curve, KFold
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.api import add_constant
from sklearn.model_selection import GridSearchCV
import warnings
import shap
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")
DATA_PATH = r"D:\Tradod\finalized dataset\finalized_dataset.csv"
TEST_SIZE = 0.2
RANDOM_STATE = 42
df_full = pd.read_csv(DATA_PATH)
df = df_full.copy()
df = df.drop([0, 1, 2, 3, 4]) 
df = df[df['year'] != 1403]
feature_cols = ['num_c1','num_c2','num_c3','num_c4','num_c5','TV_S','TV_D','TV_O','A_S','temperature','rain','province','year']
target_col = 'death'
x = df[feature_cols]
y = df[[target_col]] 
x = pd.get_dummies(x, columns=['province', 'year'])
x = x.astype(float)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
from sklearn.compose import ColumnTransformer
scale_cols = ['num_c1', 'num_c2','num_c3','num_c4','num_c5','TV_S','TV_D','TV_O','A_S','temperature','rain'] 
ct = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), scale_cols)
    ],
    remainder='passthrough'  
)
X_train_scaled = ct.fit_transform(X_train)
X_test_scaled = ct.transform(X_test)
model=ElasticNet(random_state=RANDOM_STATE, alpha=0.0003, l1_ratio=0.9, max_iter=1000, selection='random', tol=0.0001)
model.fit(X_train_scaled, y_train)

ElasticNet(alpha=0.0003, l1_ratio=0.9, random_state=42, selection='random')

# interpret_elasticnet_model func

In [18]:
def interpret_elasticnet_model(model, X_train_scaled, X_test_scaled, y_train, y_test, ct, X_train_raw, top_n=len(X_train.columns)):
    feature_names = ct.get_feature_names_out()
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_names)

    y_train = y_train.reset_index(drop=True)
    X_train_scaled_df = X_train_scaled_df.reset_index(drop=True)

    X_train_stats = sm.add_constant(X_train_scaled_df)
    ols_model = sm.OLS(y_train, X_train_stats).fit()

    print("\n📌 Constant (Intercept):", ols_model.params['const'])

    
    summary_df = ols_model.summary2().tables[1].copy()
    summary_df = summary_df.sort_values('P>|t|')

    # حذف پیشوندها برای نمایش ساده‌تر
    clean_index = summary_df.index.to_series()
    clean_index = clean_index.str.replace(r'^scaler__', '', regex=True)
    clean_index = clean_index.str.replace(r'^remainder__province_', '', regex=True)
    clean_index = clean_index.str.replace(r'^remainder__year_', '', regex=True)
    summary_df.index = clean_index

    print("\n📌 Interpretation for Traffic Violations (TV_S, TV_D, TV_O):")
    violations = ['TV_S', 'TV_D', 'TV_O']
    violations_df = summary_df.loc[summary_df.index.isin(violations)]
    print(violations_df[['Coef.', 'P>|t|', '[0.025', '0.975]']])

    print("\n📌 Interpretation for Number of Vehicles (num_c2, num_c3, num_c4, num_c5):")
    vehicle_count = ['num_c1', 'num_c2', 'num_c3', 'num_c4', 'num_c5']
    vehicle_count_df = summary_df.loc[summary_df.index.isin(vehicle_count)]
    print(vehicle_count_df[['Coef.', 'P>|t|', '[0.025', '0.975]']])

    print("\n📌 Interpretation for Average Speed (num_c2, num_c3, num_c4, num_c5):")
    vehicle_count = ['A_S']
    vehicle_count_df = summary_df.loc[summary_df.index.isin(vehicle_count)]
    print(vehicle_count_df[['Coef.', 'P>|t|', '[0.025', '0.975]']])
    
    print("\n📌 Interpretation for Weather Features (Temperature, Rain):")
    weather_features = ['temperature', 'rain']
    weather_df = summary_df.loc[summary_df.index.isin(weather_features)]
    print(weather_df[['Coef.', 'P>|t|', '[0.025', '0.975]']])

    print("\n📌 Interpretation for Provinces:")
    provinces = [col.replace("remainder__province_", "") for col in feature_names if 'province' in col]
    provinces_df = summary_df.loc[summary_df.index.isin(provinces[:top_n])]
    print(provinces_df[['Coef.', 'P>|t|', '[0.025', '0.975]']])

    print("\n📌 Interpretation for Years:")
    years = [col.replace("remainder__year_", "") for col in feature_names if 'year' in col]
    years_df = summary_df.loc[summary_df.index.isin(years[:top_n])]
    print(years_df[['Coef.', 'P>|t|', '[0.025', '0.975]']])

In [19]:
interpret_elasticnet_model(
    model=model,
    X_train_scaled=X_train_scaled,
    X_test_scaled=X_test_scaled,
    y_train=y_train,
    y_test=y_test,
    ct=ct,
    X_train_raw=X_train
)


📌 Constant (Intercept): 321.94203829675035

📌 Interpretation for Traffic Violations (TV_S, TV_D, TV_O):
          Coef.     P>|t|     [0.025     0.975]
TV_S  31.803223  0.013680   6.619109  56.987338
TV_O  -4.305340  0.483192 -16.409408   7.798728
TV_D  16.019225  0.521448 -33.243434  65.281884

📌 Interpretation for Number of Vehicles (num_c2, num_c3, num_c4, num_c5):
            Coef.     P>|t|      [0.025     0.975]
num_c2 -43.603821  0.025646  -81.826126  -5.381515
num_c3 -16.353061  0.277497  -46.003048  13.296927
num_c5  17.516979  0.371491  -21.101842  56.135800
num_c1 -33.857241  0.465336 -125.268073  57.553590
num_c4  -0.861088  0.950552  -28.256671  26.534495

📌 Interpretation for Average Speed (num_c2, num_c3, num_c4, num_c5):
         Coef.     P>|t|     [0.025    0.975]
A_S -19.772949  0.166325 -47.865421  8.319522

📌 Interpretation for Weather Features (Temperature, Rain):
                 Coef.     P>|t|     [0.025     0.975]
rain         24.169670  0.487677 -44.484364  